In [1]:
import pandas as pd
import json
import requests as rq
import base64

In [2]:
# Función get_oauth_token(): teniendo un apikey y un secret suministrados por Idealista, los guardo en un fichero settings.json como indica idealista (apikey : secret) y uso esta función para cargarlos desde el json, codificarlos en base64 (flipa) y hacer el request a la api de idealista con los otros parámetros. Una vez recibido el token, lo guardo en una varible y lo descargo en el archivo setting.json para poder recuperarlo posteriormente para hacer búsquedas (el token expira cada 43000 segundos, 12 horas)

def get_oauth_token():
    
    with open("settings.json", "r") as outfile:
        json_settings= json.load(outfile)
    message= json_settings["api_secret"]
    
    auth = "Basic " + base64.b64encode(message.encode("ascii")).decode("ascii")
    headers_dic = {"Authorization" : auth, "Content-Type" : "application/x-www-form-urlencoded;charset=UTF-8"}
    params_dic = {"grant_type" : "client_credentials", "scope" : "read"}
    content = rq.post("https://api.idealista.com/oauth/token", headers = headers_dic, params = params_dic)

    json_settings["token"]=json.loads(content.text)['access_token']
    with open('settings.json', 'w+') as outfile: 
        json.dump(json_settings,outfile)

get_oauth_token()

In [3]:
# Función search_api(URL): busca en la api de Idealista usando el token guardado en settings.json. La búsqueda hecha con sus headers y filtros se repite las veces que digamos con un for, cada json se convierte en dataframe tomando como clave "elementList" y cada dataframe (de cada iteración) se va concatenando a los anteriores.

def search_api(URL):

    with open("settings.json", "r") as outfile:
        json_settings= json.load(outfile)
    token= json_settings["token"]

    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)['access_token']
    return result


country = 'es' #values: es, it, pt
locale = 'es' #values: es, it, pt, en, ca
language = 'es' #
max_items = '50'
operation = 'sale' 
property_type = 'homes'
order = 'priceDown' 
center = '40.4167,-3.70325' 
distance = '60000'
sort = 'desc'
bankOffer = 'false'

df_tot = pd.DataFrame()
limit = 2 # max 10?

for i in range(1,limit):
    url = ('https://api.idealista.com/3.5/'+country+'/search?operation='+operation+#"&locale="+locale+
        '&maxItems='+max_items+
        '&order='+order+
        '&center='+center+
        '&distance='+distance+
        '&propertyType='+property_type+
        '&sort='+sort+ 
        '&numPage=%s'+
        '&language='+language) %(i)  
    a = search_api(URL=url)
    df = pd.DataFrame.from_dict(a["elementList"])
    df_tot = pd.concat([df_tot,df])

df_tot = df_tot.reset_index()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)